In [43]:
import tweepy
import pandas as pd
from pymongo import MongoClient, ASCENDING, TEXT

In [44]:
# read configs
consumer_key = 'jCVZcxudnaBiO8IjDIKburzU8'
consumer_secret = 'eTKxXOT1X3kGuFzpSIFewDfO0KASKcMSqfTLmiZqwFoDc8wjtH'
access_token = '1294569170135212032-yFNqooPaM1xWRwMS1dOzULaiSap2sv'
access_secret = 'Z3TFX8uNL6ulXJDLBNzPYRYCgJcikZNnheSfWSMIhgJaT'
bearer_token ='AAAAAAAAAAAAAAAAAAAAAJdtlAEAAAAA4u3gYVNv8sBwdYczEOY%2FW5hcDXo%3Duu3uKcVwFQ6bnYkTxPHIiQSkKT9fDWafMSe338rbSJzd6X305K'

In [45]:
#Connect tomongoDb Cluster
mongodb_conn=MongoClient("mongodb://localhost:27017")
db=mongodb_conn.demo # Creating a new database Demo
tweet_collection_s=db.tweet_collection # Creating a collection named tweet_collection
tweet_collection_s.create_index([('id',ASCENDING)],unique=True)
# creating an index named id on our tweet collection

'id_1'

In [46]:
# (consumer_key, consumer_secret, access_token, access_secret)
# authentication of twitter API using tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth)

In [47]:
# Listener Class
# To listen to Twitter API
class TweepyListener(tweepy.StreamingClient) : 
    # on tweet is called every time a new tweet is recieved by streaming client
    def on_tweet(self, tweet):
        keys = list(tweet.keys())
        values = list(tweet.values())
        tweet = (dict(zip(keys, values))) # dictionary collecting tweets
        print(tweet)
        try : 
            # inserting tweet into mongodb collection
            tweet_collection_s.insert_one(tweet) 
        except Exception as e: 
            print(e)
    # in case error occurs
    def on_error(self, tweet_code) : 
        if status_code == 420:
            return False


In [50]:
# Fetching Tweets
my_stream = TweepyListener("AAAAAAAAAAAAAAAAAAAAAJdtlAEAAAAA4u3gYVNv8sBwdYczEOY%2FW5hcDXo%3Duu3uKcVwFQ6bnYkTxPHIiQSkKT9fDWafMSe338rbSJzd6X305K")
# adding a new stream rule. 
# streaming tweets from CNN user in English
my_stream.add_rules(tweepy.StreamRule("CNN lang:en"))

Response(data=None, includes={}, errors=[{'value': 'CNN lang:en', 'id': '1612449611217600516', 'title': 'DuplicateRule', 'type': 'https://api.twitter.com/2/problems/duplicate-rules'}], meta={'sent': '2023-01-10T05:38:33.889Z', 'summary': {'created': 0, 'not_created': 1, 'valid': 0, 'invalid': 1}})

In [51]:
# Start streaming tweets
my_stream.filter()

{'edit_history_tweet_ids': [1612685298823036928], 'id': 1612685298823036928, 'text': "RT @catwheezie: There is no sleeping in when there's a cat to feed 💀 https://t.co/Xa77HD8IUT"}
{'edit_history_tweet_ids': [1612685302350356481], 'id': 1612685302350356481, 'text': 'RT @littlethunderr: 😼It’s called the PRIMORDIAL POUCH\naka Cat Belly Flap / 戰鬥肚 https://t.co/pXdDLYRvVf'}
{'edit_history_tweet_ids': [1612685300219940867], 'id': 1612685300219940867, 'text': 'RT @TheTweetOfJohn: Special counsel Jack Smith’s team has subpoenaed Rudy Giuliani, asking him to turn over records to a federal grand jury…'}
{'edit_history_tweet_ids': [1612685304586203136], 'id': 1612685304586203136, 'text': 'RT @TimFaulkner81: The GOP only cares about protecting their wealthy donors at the expense of middle class. \n\n“The CBO has confirmed what w…'}
{'edit_history_tweet_ids': [1612685302707159042], 'id': 1612685302707159042, 'text': 'RT @cat_auras: https://t.co/T1Jruv2SGZ'}
{'edit_history_tweet_ids': [161268530390

KeyboardInterrupt: 

In [60]:
# number of tweets and number of distinct users
print(tweet_collection_s.estimated_document_count())
user_cursor = tweet_collection_s.distinct('id')
print(len(user_cursor))

1345
1345


In [61]:
# Creating an index to search from our collection called text index
tweet_collection_s.create_index([('text', TEXT)], name='text_index', default_language='english')

'text_index'

In [62]:
# creating cursor list
# Find tweets including 'Pancake' and append them to list
tweet_cursor = tweet_collection_s.find({'$text': {'$search': 'CNN'}})

In [63]:
# Printing cursor data
for document in tweet_cursor:
    try:
        print('-----')
        print('Username:', document['_id'])
        print('Tweet:', document['text'])
    except:
        print('**error in encoding')
        pass

-----
Username: 63bbb62a83bb7c715f5ea319
Tweet: @GodmotherOlori @KateWilliamsme @andersoncooper @AC360 @CNN Very important. Particularly on CNN. No more 🤡
-----
Username: 63bce43c461998502fbe87d7
Tweet: RT @jossytweets: .@CNN is a dog with a bone and talking about documents that @JoeBiden had.

This is what's become of @CNN.
They're chasing…
-----
Username: 63bce450461998502fbe8833
Tweet: RT @ggreenwald: @AndersonMorbec @WalshFreedom @CNN Note the implied racism and sense of US superiority in that CNN exchange: Brazilians are…
-----
Username: 63bce450461998502fbe8830
Tweet: HLN Interview I did for #justice4jody. So excited for CNN documentary coming soon! Thank you CNN! https://t.co/O9seaZ9LZP
-----
Username: 63bc4c615fb480bf94dc4ee6
Tweet: RT @CNN: A Richneck Elementary School student joins “CNN This Morning” to describe the harrowing moment a lockdown was called at his school…
-----
Username: 63bbb60f83bb7c715f5ea307
Tweet: RT @CNN: Uncover the highs and lows of the controversial for